<a href="https://colab.research.google.com/github/Coyote-Schmoyote/movie-summary-generator-ja/blob/main/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
from tensorflow import keras

In [3]:
data = pd.read_csv("/content/drive/MyDrive/E資格機械学習プロジェクト/文章生成/movie-synopsis-ja.csv")

In [4]:
data.head()

,タイトル,あらすじ
0,ショーシャンクの空に,1947年、メイン州ポートランド。若くして銀行副頭取を務める優秀な銀行員アンドリュー・デュフ...
1,ゴッドファーザー,第二次世界大戦終戦直後の1945年。ニューヨーク五大ファミリーの一角で、最大の勢力を誇るイタ...
2,ダークナイト,素性不明の犯罪者ジョーカーに雇われた、道化師のマスクを被った男たちがゴッサム・シティの銀行を...
3,ロード・オブ・ザ・リング/王の帰還,昼なお暗いモルドールの山。フロドが眠る隣でゴラムは、自身が“愛しいしと”と呼ぶ“一つの指輪”...
4,シンドラーのリスト,1939年9月、ドイツ軍によりポーランドが占領され、ポーランドの都市クラクフもドイツ軍の占領...


In [5]:
data["あらすじ"][0]

'1947年、メイン州ポートランド。若くして銀行副頭取を務める優秀な銀行員アンドリュー・デュフレーン （アンディ）は、妻とその愛人を射殺した罪に問われる。無実を訴えるも終身刑の判決が下り、劣悪なショーシャンク刑務所への服役が決まる。ショーシャンクでは、長年服役する「調達屋」ことエリス・ボイド・レディング（レッド）が、もう何度目かとなる仮釈放の審査を受け、更生したことを訴えるがやはり却下される。レッドが落胆し部屋を出ると、アンディを含む新しい受刑者達が護送されて来る。アンディら新入り達はノートン所長とハドリー主任刑務官から脅しを含めたショーシャンク刑務所の紹介をされ、その晩に取り乱した一人の新人受刑者がハドリーから過剰暴力を受けて死んでしまう。\n孤立していたアンディはやがてレッドに声をかけ、鉱物採集の趣味のため小さなロックハンマーを注文する。それをきっかけにアンディはレッドと交友を重ね始める。他方、アンディは荒くれ者のボグズとその一味に性行為を強要され、抵抗のため常に生傷が絶えない生活が続いた。\n1949年、アンディは屋根の修理作業中、ハドリーの遺産相続問題を知り、財務経理の知識を駆使し作業仲間達へのビールと引き換えに解決策を提案する。ビールを手に入れ仲間達から尊敬される一方で、ハドリーら刑務官からも一目置かれるようになる。その後ボグズらがアンディを襲って全治1ヶ月の重傷を負わせるも、彼はハドリーに半殺しにされ、以後、アンディを襲う者はいなくなる。アンディが治療を終え自分の房に戻ってくると、レッドに注文していたリタ・ヘイワースの大判ポスターが退院祝いとして置かれていた。\nやがて、アンディは図書係に配置換えとなり、もう50年も服役している老囚人ブルックスの助手となる。だが、その本当の目的は所長や刑務官達の税務処理や資産運用をアンディに行わせるためだった。アンディは有能な銀行家としての手腕を発揮する一方で、名ばかりだった図書係としても精力的に活動を始め、州議会に図書館予算の請求を毎週送るようになる。\n1954年、ブルックスに仮釈放の許可が下りるが、50年服役した老人は塀の外の生活への恐れから取り乱す。アンディらに説得され、仮釈放を受け入れるが、結局、外の生活に馴染むことはできず、最期は首を吊って死んでしまう。死の間際に送られた感謝の手紙を読んでアンディとレッドは苛

In [26]:
data.columns = ["Title", "Summary"]

In [28]:
data.columns

Index(['Title', 'Summary'], dtype='object')

In [30]:
data.isnull().sum()

Title      0
Summary    0
dtype: int64

In [8]:
data = data.dropna(axis=0)

In [29]:
data.isnull().sum()

Title      0
Summary    0
dtype: int64

In [10]:
data.shape

(99, 2)

In [31]:
data.dtypes

Title      object
Summary    object
dtype: object

## Data Processing

In [12]:
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.3 MB/s 


In [13]:
#dictionary
!pip install unidic-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.4 MB 1.8 MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=16174d531acb8addebd0b85c6f74410aec4add583be6b4d95cc309783bc1c451
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


In [36]:
!pip install spacy[ja]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.2 MB/s 
  Created wheel for sudachidict-core: filename=SudachiDict_core-20220729-py3-none-any.whl size=71570387 sha256=6d8318d53a2543ef006a3b0afbd7d92a06514aaff2c79121e870b9f1db9bd4d2
  Stored in directory: /root/.cache/pip/wheels/ff/47/2b/6dcb905d9086fc6f9b7af821f7145f785d36070b827b7f80b7
Successfully built sudachidict-core


In [41]:
!pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.7 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172952 sha256=46088bb662c39691515ca453d586106fe32c8994c58631f2335d49219a0ef2d3
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn


In [50]:
!pip install -U ginza ja-ginza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 104 kB/s 
  Attempting uninstall: ginza
    Found existing installation: ginza 5.1.1
    Uninstalling ginza-5.1.1:
      Successfully uninstalled ginza-5.1.1


In [43]:
import MeCab
import spacy
import neologdn
from spacy.lang.ja import Japanese

In [15]:
wakati = MeCab.Tagger("-Owakati")

In [32]:
review = wakati.parse(data["Title"][0]).split()

In [25]:
review[:50]

['1947',
 '年',
 '、',
 'メイン',
 '州',
 'ポートランド',
 '。',
 '若く',
 'し',
 'て',
 '銀行',
 '副',
 '頭取',
 'を',
 '務める',
 '優秀',
 'な',
 '銀行',
 '員',
 'アンドリュー・デュフレーン',
 '（',
 'アンディ',
 '）',
 'は',
 '、',
 '妻',
 'と',
 'その',
 '愛人',
 'を',
 '射殺',
 'し',
 'た',
 '罪',
 'に',
 '問わ',
 'れる',
 '。',
 '無実',
 'を',
 '訴える',
 'も',
 '終身',
 '刑',
 'の',
 '判決',
 'が',
 '下り',
 '、',
 '劣悪']

In [22]:
def clean(data):
  data = BeautifulSoup(data).get_text()
  data = wakati.parse(data).split()
  return data


In [44]:
summary = []
for i in range(len(data["Summary"])):
  summary.append(clean(data["Summary"][i]))

In [45]:
summary[1][:50]

['第',
 '二',
 '次',
 '世界',
 '大戦',
 '終戦',
 '直後',
 'の',
 '1945',
 '年',
 '。',
 'ニューヨーク',
 '五',
 '大',
 'ファミリー',
 'の',
 '一角',
 'で',
 '、',
 '最大',
 'の',
 '勢力',
 'を',
 '誇る',
 'イタリア',
 '系',
 'マフィア',
 '「',
 'コルレオーネ',
 '・',
 'ファミリー',
 '」',
 'の',
 '邸宅',
 'で',
 'は',
 '、',
 'ドン',
 '・',
 'コルレオーネ',
 '（',
 'ヴィトー',
 '）',
 'の',
 '娘',
 'コニー',
 'の',
 '結婚',
 '式',
 'が']

In [54]:
def ja_preprocess(text):
    nlp = spacy.load('ja_ginza')
    lowered = text.lower()
    normalized = neologdn.normalize(lowered)
    tokenized = []
    for i in nlp(normalized):
        word = i.lemma_
        pos = i.pos_
        if pos in ["NOUN", "ADJ", "VERB", "ADV", "PROPN"] and len(word) > 1 and i.text:
            tokenized.append(word)
    preprocessed = " ".join(tokenized)
    return preprocessed

In [55]:
summary = data["Summary"]